In [1]:
import pandas as pd
import requests
from io import BytesIO
import zipfile
import os

# Informações os dados a serem baixados

### Rotina para buscar URL do World Uncertainty Index

In [2]:
import requests
from bs4 import BeautifulSoup

# Target page
url = "https://worlduncertaintyindex.com/data/"

# Fetch the page content
response = requests.get(url, verify=False)
response.raise_for_status()

# Parse HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find the link
World_Uncertainty_Index = None
for link in soup.find_all('a', href=True):
    if "World Uncertainty Index: Monthly Dataset" in link.text:
        World_Uncertainty_Index = link['href']
        break



c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worlduncertaintyindex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### Cria dicionário para busca dos arquivos de Excel

In [4]:
fontes = [
    {
        "Indicador": "Trade Policy Uncertainty",
        "Link": "https://www.matteoiacoviello.com/tpu_files/tpu_web_latest.xlsx",
        "Excel_sheet": "TPU_MONTHLY",
        "Column_excel_sheet": ["A","E"]
    },
    {
        "Indicador": "World Uncertainty Index",
        "Link": World_Uncertainty_Index, #"https://worlduncertaintyindex.com/wp-content/uploads/2025/09/WUI_M_dataset_2025_08.xlsx"
        "Excel_sheet": "F1",
        "Column_excel_sheet": ["A", "B"]
    },
    {
        "Indicador": "Global Economic Policy Uncertainty Index",
        "Link": "https://www.policyuncertainty.com/media/Global_Policy_Uncertainty_Data.xlsx",
        "Excel_sheet": "Sheet1",
        "Column_excel_sheet": ["A", "B", "D"]
    }
]

# Downloading files

In [5]:
def download_all_links(fontes):
    os.makedirs("00.data/01.incertezas", exist_ok=True)
    for fonte in fontes:
        try:
            resp = requests.get(fonte["Link"], verify=False)
            filename = os.path.join("00.data/01.incertezas", fonte["Link"].split("/")[-1])
            with open(filename, "wb") as f:
                f.write(resp.content)
            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Failed to download {fonte['Indicador']}: {e}")

download_all_links(fontes)

c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.matteoiacoviello.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: 00.data/01.incertezas\tpu_web_latest.xlsx


c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'worlduncertaintyindex.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: 00.data/01.incertezas\WUI_M_dataset_2025_10.xlsx


c:\Users\F3189218\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.policyuncertainty.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded: 00.data/01.incertezas\Global_Policy_Uncertainty_Data.xlsx


# Leitura de cada arquivo Excel separadamente e salvando em dataframes distintos

In [30]:

# Trade Policy Uncertainty
trade_policy_uncertainty_df = pd.read_excel(
    "00.data/01.incertezas/TPU_web_latest.xlsx",
    sheet_name="TPU_MONTHLY",
    usecols="A,E",
    engine="openpyxl"
)
trade_policy_uncertainty_df.columns = ["Date", "Trade Policy Uncertainty"]
trade_policy_uncertainty_df["Date"] = pd.to_datetime(trade_policy_uncertainty_df["Date"], errors="coerce")
trade_policy_uncertainty_df = trade_policy_uncertainty_df.dropna(subset=["Date"]).set_index("Date")


In [6]:
world_uncertainty_index_excel = World_Uncertainty_Index.split("/")[-1]

# World Uncertainty Index
world_uncertainty_index_df = pd.read_excel(
    os.path.join("00.data/01.incertezas/", world_uncertainty_index_excel),
    sheet_name="F1",
    usecols="A,B",
    engine="openpyxl"
)
world_uncertainty_index_df.columns = ["Date", "World Uncertainty Index"]
world_uncertainty_index_df["Date"] = pd.to_datetime(world_uncertainty_index_df["Date"], errors="coerce")
world_uncertainty_index_df = world_uncertainty_index_df.dropna(subset=["Date"]).set_index("Date")


C:\Users\F3189218\AppData\Local\Temp\ipykernel_9900\691704653.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  world_uncertainty_index_df["Date"] = pd.to_datetime(world_uncertainty_index_df["Date"], errors="coerce")


In [7]:

# Global Economic Policy Uncertainty Index
global_economic_policy_uncertainty_df = pd.read_excel(
    "00.data/01.incertezas/Global_Policy_Uncertainty_Data.xlsx",
    sheet_name="Sheet1",
    usecols="A,B,D",
    engine="openpyxl"
)
global_economic_policy_uncertainty_df.columns = [
    "year",
    "month",
    "Global Economic Policy Uncertainty Index"
]
# Convert year and month to datetime
global_economic_policy_uncertainty_df["Date"] = pd.to_datetime(
    global_economic_policy_uncertainty_df["year"].astype(str) + "-" +
    global_economic_policy_uncertainty_df["month"].fillna(1).astype(int).astype(str) + "-01",
    errors="coerce"
)


# Merge dataframes

In [10]:
merged_df = global_economic_policy_uncertainty_df.set_index("Date")[
    ["Global Economic Policy Uncertainty Index"]
].join([
    world_uncertainty_index_df,
    trade_policy_uncertainty_df
], how="outer")

NameError: name 'trade_policy_uncertainty_df' is not defined

In [11]:
merged_df.to_excel("00.data/01.incertezas/merged_uncertainty_indices.xlsx")

NameError: name 'merged_df' is not defined